## Fibonacci

In [1]:
def fib(n):
    if n<2:
        return n
    return fib(n-1)+fib(n-2)

In [2]:
%timeit fib(20)

100 loops, best of 3: 9.77 ms per loop


### Cython

In [3]:
%load_ext Cython

In [4]:
%%cython
def fib_cython(n):
    if n<2:
        return n
    return fib_cython(n-1)+fib_cython(n-2)

In [5]:
%timeit fib_cython(20)

100 loops, best of 3: 3.06 ms per loop


In [6]:
%%cython
cpdef long fib_cython_type(long n):
    if n<2:
        return n
    return fib_cython_type(n-1)+fib_cython_type(n-2)

In [7]:
%timeit fib_cython_type(20)

10000 loops, best of 3: 128 µs per loop


### Caching

In [8]:
from functools import lru_cache as cache

@cache(maxsize=None)
def fib_cache(n):
    if n<2:
        return n
    return fib_cache(n-1)+fib_cache(n-2)

In [9]:
%timeit fib_cache(20)

The slowest run took 206.64 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 3: 147 ns per loop


In [11]:
def fib_seq(n):
    if n < 2:
        return n
    a,b = 1,0
    for i in range(n-1):
        a,b = a+b,a
    return a   

In [12]:
%timeit fib_seq(20)

1000000 loops, best of 3: 1.45 µs per loop


In [13]:
%%cython

def fib_seq_cython(n):
    if n < 2:
        return n
    a,b = 1,0
    for i in range(n-1):
        a,b = a+b,a
    return a    

cpdef long fib_seq_cython_type(long n):
    if n < 2:
        return n
    cdef long a,b
    a,b = 1,0
    for i in range(n-1):
        a,b = a+b,a
    return a

In [14]:
%timeit fib_seq_cython(20)
%timeit fib_seq_cython_type(20)

1000000 loops, best of 3: 810 ns per loop
10000000 loops, best of 3: 73.3 ns per loop


### Compiling with Numba

In [15]:
from numba import jit

In [16]:
@jit
def fib_seq_numba(n):
    if n < 2:
        return n
    a,b = 1,0
    for i in range(n-1):
        a,b = a+b,a
    return a   

In [17]:
%timeit fib_seq_numba(20)

The slowest run took 839660.89 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 3: 428 ns per loop


## Quicksort

### Numpy

In [18]:
def qsort_kernel(a, lo, hi):
    i = lo
    j = hi
    while i < hi:
        pivot = a[(lo+hi) // 2]
        while i <= j:
            while a[i] < pivot:
                i += 1
            while a[j] > pivot:
                j -= 1
            if i <= j:
                a[i], a[j] = a[j], a[i]
                i += 1
                j -= 1
        if lo < j:
            qsort_kernel(a, lo, j)
        lo = i
        j = hi
    return a

In [19]:
import random
def benchmark_qsort():
    lst = [ random.random() for i in range(1,5000) ]
    qsort_kernel(lst, 0, len(lst)-1)

In [20]:
%timeit benchmark_qsort()

10 loops, best of 3: 38.6 ms per loop


In [21]:
%%cython
import numpy as np
import cython

@cython.boundscheck(False)
@cython.wraparound(False)
cdef double[:] \
qsort_kernel_cython_numpy_type(double[:] a, \
                               long lo, \
                               long hi):
    cdef: 
        long i, j
        double pivot
    i = lo
    j = hi
    while i < hi:
        pivot = a[(lo+hi) // 2]
        while i <= j:
            while a[i] < pivot:
                i += 1
            while a[j] > pivot:
                j -= 1
            if i <= j:
                a[i], a[j] = a[j], a[i]
                i += 1
                j -= 1
        if lo < j:
            qsort_kernel_cython_numpy_type(a, lo, j)
        lo = i
        j = hi
    return a

def benchmark_qsort_numpy_cython():
    lst = np.random.rand(5000)
    qsort_kernel_cython_numpy_type(lst, 0, len(lst)-1)

In [22]:
%timeit benchmark_qsort_numpy_cython()

The slowest run took 5.44 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 1.03 ms per loop


In [23]:
def benchmark_sort_numpy():
    lst = np.random.rand(5000)
    np.sort(lst)

In [24]:
%timeit benchmark_sort_numpy()

The slowest run took 4.47 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 319 µs per loop


## Mandelbrot Set

In [25]:
def mandel(z):
    maxiter = 80
    c = z
    for n in range(maxiter):
        if abs(z) > 2:
            return n
        z = z*z + c
    return maxiter

def mandelperf():
    r1 = np.linspace(-2.0, 0.5, 26)
    r2 = np.linspace(-1.0, 1.0, 21)
    return [mandel(complex(r, i)) for r in r1 for i in r2]
 
assert sum(mandelperf()) == 14791

In [26]:
%timeit mandelperf()

100 loops, best of 3: 3.89 ms per loop


### Cython

In [28]:
%%cython
def mandel_cython(z):
    maxiter = 80
    c = z
    for n in range(maxiter):
        if abs(z) > 2:
            return n
        z = z*z + c
    return maxiter

In [29]:
def mandelperf_cython():
    r1 = np.linspace(-2.0, 0.5, 26)
    r2 = np.linspace(-1.0, 1.0, 21)
    return [mandel_cython(complex(r, i)) for r in r1 for i in r2]
 
assert sum(mandelperf_cython()) == 14791

In [30]:
%timeit mandelperf_cython()

1000 loops, best of 3: 1.69 ms per loop


### Numba

In [48]:
@jit
def mandel_numba(z):
    maxiter = 80
    c = z
    for n in range(maxiter):
        if abs(z) > 2:
            return n
        z = z*z + c
    return maxiter

def mandelperf_numba():
    r1 = np.linspace(-2.0, 0.5, 26)
    r2 = np.linspace(-1.0, 1.0, 21)
    return [mandel_numba(complex(r, i)) for r in r1 for i in r2]

In [49]:
%timeit mandelperf_numba()

The slowest run took 147.72 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 1.17 ms per loop


### Profiling

In [33]:
%load_ext line_profiler

In [50]:
run = %lprun -r -s -f mandelperf_numba mandelperf_numba()
run.print_stats()

Timer unit: 1e-06 s

Total time: 0.000784 s
File: <ipython-input-48-621342073929>
Function: mandelperf_numba at line 11

Line #      Hits         Time  Per Hit   % Time  Line Contents
    11                                           def mandelperf_numba():
    12         1           82     82.0     10.5      r1 = np.linspace(-2.0, 0.5, 26)
    13         1           35     35.0      4.5      r2 = np.linspace(-1.0, 1.0, 21)
    14         1          667    667.0     85.1      return [mandel_numba(complex(r, i)) for r in r1 for i in r2]



In [51]:
def mandelperf_numba():
    r1 = np.linspace(-2.0, 0.5, 26)
    r2 = np.linspace(-1.0, 1.0, 21)
    c3 = [complex(r,i) for r in r1 for i in r2]
    return [mandel_numba(c) for c in c3]

In [57]:
%timeit mandelperf_numba()

1000 loops, best of 3: 429 µs per loop


In [52]:
run = %lprun -r -s -f mandelperf_numba mandelperf_numba()
run.print_stats()

Timer unit: 1e-06 s

Total time: 0.001145 s
File: <ipython-input-51-f5e8d196d72f>
Function: mandelperf_numba at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def mandelperf_numba():
     2         1          110    110.0      9.6      r1 = np.linspace(-2.0, 0.5, 26)
     3         1           53     53.0      4.6      r2 = np.linspace(-1.0, 1.0, 21)
     4         1          527    527.0     46.0      c3 = [complex(r,i) for r in r1 for i in r2]
     5         1          455    455.0     39.7      return [mandel_numba(c) for c in c3]



In [53]:
@jit
def mandelperf_numba_mesh():
    width = 26
    height = 21
    r1 = np.linspace(-2.0, 0.5, width)
    r2 = np.linspace(-1.0, 1.0, height)
    mandel_set = np.empty((width,height), dtype=int)
    for i in range(width):
        for j in range(height):
            mandel_set[i,j] = mandel_numba(r1[i] + 1j*r2[j])
    return mandel_set

In [55]:
%timeit mandelperf_numba_mesh()

10000 loops, best of 3: 129 µs per loop


In [ ]:
# def mandel_numba(z):
#     maxiter = 80
#     c = z
#     for n in range(maxiter):
#         if abs(z) > 2:
#             return n
#         z = z*z + c
#     return maxiter

def mandelperf_vector_mesh():
    width = 26
    height = 21
    r1 = np.linspace(-2.0, 0.5, width)
    r2 = np.linspace(-1.0, 1.0, height)
    mandel_set = np.empty((width,height), dtype=int)
    
    
    
    return mandel_set

### Vectorizing